<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Telegram_Search_Source.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q telethon python-dotenv nest_asyncio

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.2/702.2 kB 12.6 MB/s eta 0:00:00


In [3]:
import asyncio
import json
import os
from telethon.sync import TelegramClient
from telethon.sessions import StringSession
from telethon.tl.types import MessageMediaPhoto, MessageMediaWebPage, MessageMediaDocument
from dotenv import load_dotenv
import nest_asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Load environment variables
load_dotenv()

# Telegram API Keys
TELEGRAM_API_ID = 20441646
TELEGRAM_API_HASH = "d78a891287e9ba6a2a8c4bb0e4ca506a"

In [4]:
# Telegram Channels to Scrape
TELEGRAM_CHANNELS = [
    'cveNotify',
    'ctinow',
    'CyberSecurityTechnologies',
    'cybersecurity_outlook',
    'cibsecurity',
    'thehackernews',
    'Cyber_Security_Channel',
    'cloudandcybersecurity',
    'androidMalware',
    'DarkfeedNews',
    'PentestingNews',
    'malwr',
    'CyberSecurityNews',
    'CyberThreatIntel',
    'CyberDefenseMagazine',
    'CyberArk',
    'CyberScoopNews',
    'CyberWire',
    'CyberSecurityDaily',
    'CyberSecurityUpdates',
    'CyberSecurityAlerts',
    'CyberSecurityBreach',
    'CyberSecurityIncidents',
    'CyberSecurityThreats',
    'CyberSecurityVulnerabilities',
    'CyberIncidentAlerts',
    'CyberAttackNews',
    'CyberBreachReports',
    'CyberIncidentResponse',
    'CyberThreatNews',
    'CyberIncidentUpdates',
    'CyberSecurityIncidentReports',
    'CyberIncidentNotifications',
    'CyberIncidentAlertsGlobal',
    'CyberIncidentNewsFeed',
    'vxunderground',
    'ransomwatcher',
    'SecurityWeekly',
    'BleepingComputer',
    'RecordedFuture',
    'RapidCyberAlerts',
    'KrebsOnSecurity',
    'SOCRadar',
    'RedAlertCyber',
    'ZeroDay_TI',
    'AT_Hack',
    'OSINT_Tactical',
    'malware_traffic',
    'vulnerability_lab',
    'ThreatIntelligence',
    'CERTNews',
    'DataBreachToday',
    'InfoSecNews',
    'APTHunter',
    'MalwarePatrol',
    'DigitalSecurityPulse',
    'ExploitWareLabs',
    'CyberSecMatrix',
    'ThreatFeed',
    'SecurityAffairs',
    'RansomwareTracker',
    'PhishingAlerts',
    'MITREUpdates',
    'DarkWebMonitor',
    'CriticalVulns',
    'CloudSecurityNews',
    'IOCFeed',
    'ThreatHunting',
    'MalwareIntel',
    'IncidentResponse'
]

In [5]:
# File to store the session string
SESSION_FILE = "telegram_session.txt"

def generate_session_string():
    """
    Generate a new session string and save it to a file.
    """
    print("Generating a new session string...")
    with TelegramClient(StringSession(), TELEGRAM_API_ID, TELEGRAM_API_HASH) as client:
        session_string = client.session.save()
        with open(SESSION_FILE, "w") as f:
            f.write(session_string)
        print(f"Session string saved to {SESSION_FILE}")
    return session_string

def load_session_string():
    """
    Load the session string from the file. If the file doesn't exist, generate a new session string.
    """
    if os.path.exists(SESSION_FILE):
        with open(SESSION_FILE, "r") as f:
            return f.read().strip()
    else:
        return generate_session_string()

# Initialize Telegram client with StringSession
session_string = load_session_string()
telegram_client = TelegramClient(StringSession(session_string), TELEGRAM_API_ID, TELEGRAM_API_HASH)

async def start_telegram_client():
    """
    Start the Telegram client using the saved session string.
    If the session is invalid, prompt for OTP verification.
    """
    try:
        await telegram_client.connect()
        if not await telegram_client.is_user_authorized():
            print("Session is invalid. Please log in again.")
            await telegram_client.start(phone=lambda: input('Please enter your phone number: '))
            if not await telegram_client.is_user_authorized():
                await telegram_client.send_code_request(input('Please enter your phone number: '))
                await telegram_client.sign_in(input('Please enter the code you received: '))
            # Save the new session string
            with open(SESSION_FILE, "w") as f:
                f.write(telegram_client.session.save())
            print("New session string saved.")
        print("Telegram client started successfully")
    except Exception as e:
        print(f"Error starting Telegram client: {e}")

async def scrape_telegram_channel(channel_name: str):
    """
    Scrape messages from a Telegram channel.
    """
    try:
        print(f"Scraping channel: {channel_name}")
        messages = []
        async for message in telegram_client.iter_messages(channel_name, limit=100):  # Limit to 100 messages for testing
            media_urls = []
            media_content = []
            # Check if message has media and handle various media types
            if hasattr(message, 'media') and message.media:
                if isinstance(message.media, MessageMediaPhoto):
                    media_path = await telegram_client.download_media(message.media, file=f"./media/{channel_name}/{message.id}.jpg")
                    media_urls.append(media_path)
                    media_content.append({"image_url": media_path})
                elif isinstance(message.media, MessageMediaWebPage):
                    if hasattr(message.media.webpage, 'url'):
                        media_urls.append(message.media.webpage.url)
                        media_content.append({"webpage_url": message.media.webpage.url})
                elif isinstance(message.media, MessageMediaDocument):
                    media_path = await telegram_client.download_media(message.media, file=f"./media/{channel_name}/{message.id}.doc")
                    media_urls.append(media_path)
                    media_content.append({"document_url": media_path})
                else:
                    print(f"Unhandled media type: {type(message.media)}")

            messages.append({
                'id': message.id,
                'date': message.date.isoformat(),
                'message': message.message,
                'media_urls': media_urls,
                'media_content': media_content,
                'views': message.views,
                'forwards': message.forwards
            })

        return messages
    except Exception as e:
        print(f"Error scraping channel {channel_name}: {e}")
        return []

Generating a new session string...


RuntimeError: You must use "async with" if the event loop is running (i.e. you are inside an "async def")

In [6]:
async def scrape_all_channels(channels: list):
    """
    Scrape messages from all specified Telegram channels.
    """
    tasks = [scrape_telegram_channel(channel) for channel in channels]
    return await asyncio.gather(*tasks)

def save_to_json(data, filename):
    """
    Save scraped data to a JSON file.
    """
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)
    print(f"Data saved to {filename}")

def get_telegram_channel_data(channels: list):
    """
    Get data from all specified Telegram channels.
    """
    all_telegram_data = asyncio.run(scrape_all_channels(channels))
    return all_telegram_data

In [7]:
# Main execution
if __name__ == "__main__":
    # Ensure media directory exists
    os.makedirs("./media", exist_ok=True)
    for channel in TELEGRAM_CHANNELS:
        os.makedirs(f"./media/{channel}", exist_ok=True)

    # Start the Telegram client
    asyncio.run(start_telegram_client())

    # Get data from all specified Telegram channels
    all_telegram_data = get_telegram_channel_data(TELEGRAM_CHANNELS)

    # Flatten the list of lists
    all_telegram_data = [item for sublist in all_telegram_data for item in sublist]

    # Save data to JSON
    save_to_json(all_telegram_data, 'telegram_data.json')

    print("Scraping completed successfully")

NameError: name 'start_telegram_client' is not defined